In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# ========== python ==========
import os
from pathlib import Path
from logging import Logger
from typing import List, Dict, Tuple, Optional, Union, Callable, Final, Literal, get_args
from operator import itemgetter, attrgetter
import itertools
from IPython.display import display

from utils.setup import setup_logger, get_device
from const.const_values import PROJECT_DIR

os.chdir(PROJECT_DIR)
logger: Logger = setup_logger(__name__, f'{PROJECT_DIR}/log/jupyter_run.log')
device = get_device(device_name='cpu', logger=logger)

In [2]:
# jupyter
import seaborn as sns
import matplotlib.pyplot as plt
# Machine learning
import numpy as np
import pandas as pd
import h5py
import optuna
# torch
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard.writer import SummaryWriter
# torch ignite
from ignite.engine import Engine
from ignite.handlers import Checkpoint
# My items
from models.datasets.data_helper import MyDataHelperForStory, MyDataLoaderHelper, DefaultTokens
from models.datasets.datasets_for_story import StoryTriple
# My utils
from utils.setup import load_param
from utils.torch import load_model, torch_fix_seed
# main function
from run_for_KGC import main_function


In [3]:
from const.const_values import CPU, MODEL
from models.KGModel.kg_model import HEAD, RELATION, TAIL
from utils.torch_ignite import TRAINER, EVALUATOR
from const.const_values import DATASETS, DATA_HELPER, DATA_LOADERS, TRAIN_RETURNS

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [5]:
SEED: Final[int] = 42
args_path = f'{PROJECT_DIR}/models/kgc03/SpeckledBand090/param.pkl'
model_path = f'{PROJECT_DIR}/models/kgc03/SpeckledBand090/model.pth'

In [6]:
args = load_param(args_path)

# args.pre_train = True
args.logger = logger
args.device = device
args.batch_size = 1
args.pre_train=False
args.init_embedding_using_bert = False
args.model_path = model_path
args.only_load_trainer_evaluator = True

del args.optuna_file, args.device_name, args.pid, args.study_name, args.n_trials
logger.info(args)

2023-02-03 22:53:25 - INFO - 929291086.py - 13 - Namespace(notebook=False, console_level='info', logfile='models/kgc03/SpeckledBand090/log.log', param_file='models/kgc03/SpeckledBand090/param.pkl', train_anyway=False, old_data=0, tensorboard_dir='models/kgc03/SpeckledBand090/tensorboard', checkpoint_dir='models/kgc03/SpeckledBand090/checkpoint/', model_path='/Users/ryoyakaneda/Documents/学校/M1Study/knowledge_graph/models/kgc03/SpeckledBand090/model.pth', resume_from_checkpoint=False, resume_from_last_point=False, only_load_trainer_evaluator=True, resume_checkpoint_path=None, pre_train=False, train_valid_test=False, only_train=True, use_for_challenge100=False, use_for_challenge090=True, use_for_challenge075=False, use_title='SpeckledBand', do_optuna=False, story_special_num=5, relation_special_num=5, entity_special_num=5, padding_token_e=0, cls_token_e=1, mask_token_e=2, sep_token_e=3, bos_token_e=4, padding_token_r=0, cls_token_r=1, mask_token_r=2, sep_token_r=3, bos_token_r=4, padding_

In [7]:
torch_fix_seed(seed=SEED)
return_dict = main_function(args, logger=logger)

2023-02-03 22:53:25 - INFO - run_for_KGC.py - 893 - ----- make datahelper start. -----
2023-02-03 22:53:25 - INFO - data_helper.py - 333 - ====== Show MyRawData ======
2023-02-03 22:53:25 - INFO - data_helper.py - 334 - entity num: 7812
2023-02-03 22:53:25 - INFO - data_helper.py - 335 - relation num: 62
2023-02-03 22:53:25 - INFO - data_helper.py - 336 - ====== Show MyRawData ======
2023-02-03 22:53:25 - INFO - data_helper.py - 607 - ========== Show DataHelper ==========
2023-02-03 22:53:25 - INFO - data_helper.py - 333 - ====== Show MyRawData ======
2023-02-03 22:53:25 - INFO - data_helper.py - 334 - entity num: 7812
2023-02-03 22:53:25 - INFO - data_helper.py - 335 - relation num: 62
2023-02-03 22:53:25 - INFO - data_helper.py - 336 - ====== Show MyRawData ======
2023-02-03 22:53:25 - INFO - data_helper.py - 609 - entity_special_dicts: {0: '<pad_e>', 1: '<cls_e>', 2: '<mask_e>', 3: '<sep_e>', 4: '<bos_e>'}
2023-02-03 22:53:25 - INFO - data_helper.py - 610 - relation_special_dicts: {

In [8]:
model = return_dict[MODEL]

dataset_train: StoryTriple = return_dict[DATASETS][0]
triple: torch.Tensor = dataset_train.triple
data_helper: MyDataHelperForStory = return_dict[DATA_HELPER]
evaluator: Checkpoint = return_dict[TRAIN_RETURNS][EVALUATOR]

load_model(model, args.model_path, device)
model.eval()

entities, relations = data_helper.processed_entities, data_helper.processed_relations
d_e, d_r = {e: i for i, e in enumerate(entities)}, {r: i for i, r in enumerate(relations)}

triple_df = pd.DataFrame([(entities[_t[0]], relations[_t[1]], entities[_t[2]]) for _t in triple], columns=[HEAD, RELATION, TAIL])
story_entities = triple_df[HEAD].tolist()

In [9]:
triple_df

,head,relation,tail
0,<bos_e>,<bos_r>,<bos_e>
1,AbbeyGrange:001,kgc:subject,AllTitle:Holmes
2,AbbeyGrange:001,kgc:hasPredicate,word.predicate:wakeUp
3,AbbeyGrange:001,kgc:whom,AllTitle:Watson
4,AbbeyGrange:001,kgc:infoSource,AllTitle:Watson
...,...,...,...
17658,AbbeyGrange:084,kgc:subject,AbbeyGrange:Theresa
17659,AbbeyGrange:084,kgc:hasPredicate,word.predicate:rest
17660,AbbeyGrange:084,kgc:where,AbbeyGrange:room_of_Lady_Brackenstall
17661,<bos_e>,<bos_r>,<bos_e>


In [16]:
triple_df[triple_df['head'].str.contains('SpeckledBand')]

,head,relation,tail
15001,SpeckledBand:001,kgc:subject,SpeckledBand:Helen
15002,SpeckledBand:001,kgc:hasPredicate,word.predicate:arrive
15003,SpeckledBand:001,kgc:to,SpeckledBand:house_of_Holmes
15004,SpeckledBand:001,kgc:when,SpeckledBand:1883-04-01T07
15005,SpeckledBand:001,kgc:time,DateTime:1883-04-01T07:00:00
...,...,...,...
17146,SpeckledBand:360,kgc:subject,AllTitle:Watson
17147,SpeckledBand:360,kgc:hasPredicate,word.predicate:approach
17148,SpeckledBand:360,kgc:to,SpeckledBand:Roylott
17149,SpeckledBand:360,kgc:when,SpeckledBand:1883-04-02T04


In [10]:
def extract(_model, target, inputs):
    features = None

    def forward_hook(_module, _inputs, _):
        nonlocal features
        x, _, _ = _inputs
        outputs = _module.forward(x, x, x, need_weights=True)[1]
        features = outputs.detach().clone()

    handle = target.register_forward_hook(forward_hook)

    _model.eval()
    _model(inputs, torch.LongTensor([[]]), torch.LongTensor([[]]), torch.LongTensor([[]]))

    handle.remove()

    return features

def get_attention(input_):
    assert len(input_) == 1
    features = extract(model, model.transformer.layers[-1].self_attn, input_)[0]
    df_attention = pd.DataFrame([[entities[h], relations[r], entities[t]]+[features[j, i].item() for j in range(len(features))] for i, (h, r, t) in enumerate(input_[0])])
    df_attention.columns=[HEAD, RELATION, TAIL] + [f'atten_from{i}' for i in range(len(df_attention.columns)-3)]
    return df_attention

def show_attension_heatmap(df_attention):
    sns.heatmap(df_attention.iloc[:,3:])
    plt.show()

In [11]:
MASK_E = DefaultTokens.MASK_E
KILL = 'word.predicate:kill'

TAKE = 'word.predicate:take'
BRING = 'word.predicate:bring'
DIE = 'word.predicate:die'
HIDE = 'word.predicate:hide'

In [12]:
bos_triple = [d_e[DefaultTokens.BOS_E], d_r[DefaultTokens.BOS_R],d_e[DefaultTokens.BOS_E]]
mask_e_id = d_e[DefaultTokens.MASK_E]
Holmes_id =d_e['AllTitle:Holmes']

def make_ranking(from_story_name, to_story_name, predicate_, whom_, subject_, why_, what_, where_):
    if not (from_story_name is None and to_story_name is None):
        _start_index = story_entities.index(from_story_name)-1 
        _end_index = len(story_entities) - story_entities[::-1].index(to_story_name)
    else: 
        _start_index = 0
        _end_index = 0
    question_ = torch.tensor(
        [
            bos_triple, 
            [mask_e_id, d_r['kgc:infoSource'],     Holmes_id      ],
            [mask_e_id, d_r['kgc:hasPredicate'],   d_e[predicate_]],
            [mask_e_id, d_r['kgc:whom'],           d_e[whom_     ]],
            [mask_e_id, d_r['kgc:subject'],        d_e[subject_  ]],
            [mask_e_id, d_r['kgc:why'],            d_e[why_      ]],
            [mask_e_id, d_r['kgc:what'],           d_e[what_     ]],
            [mask_e_id, d_r['kgc:where'],          d_e[where_    ]],
        ]
    )
    mask_ = torch.zeros_like(question_, dtype=torch.bool) # not mask all position
    mask_[1:, 0] = True                                   # where head position without bos token
    mask_[1:, 2] = True                                   # where tail position without bos token

    last_triples = triple[_start_index: _end_index]

    questions = torch.cat([last_triples, question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros_like(last_triples), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)

    data_list = []
    with torch.no_grad():
        _, (story_pred, relation_pred, entity_pred) = model(questions, masks[:,0], masks[:,1], masks[:,2])
        sorted_ = torch.argsort(entity_pred, dim=1, descending=True)
        for i in range(sorted_.shape[1]):
            ans_= sorted_[:, i]
            info_source_, predicate_pred, whom_pred, subject_pred, why_pred, what_pred, where_pred = ans_
            data_list.append([entities[predicate_pred], entities[whom_pred], entities[subject_pred], entities[why_pred], entities[what_pred], entities[where_pred]])
    df_ranking = pd.DataFrame(data_list, columns=['predicate', 'whom', 'subject', 'why', 'what', 'where'])
    df_attension = get_attention(questions)
    
    return df_ranking, df_attension

In [13]:
def main_func01(_title, _victim_name, criminal, predicate, _last_index, _story_len):
    from_ = f'{_title}:{_last_index-_story_len+1}'
    to_ = f'{_title}:{_last_index}'
    predicate = predicate
    victim = f'{_title}:{_victim_name}'
    criminal = f'{_title}:{criminal}'
    df_ranking, df_attension = make_ranking(
        from_, to_, predicate, victim, MASK_E, MASK_E, MASK_E, MASK_E)

    pred_rank = df_ranking.index[df_ranking['subject']==criminal].tolist()
    pred_rank = pred_rank[0] if len(pred_rank)==1 else -1
    logger.info(f"The pred ranking about {criminal} is {pred_rank}")
    display(df_ranking.iloc[:max(20, pred_rank)])
    len_ = len(df_attension)
    for i in range(len_-10, len_):
        print(f"index={i}, triple={df_attension.iloc[i,:3].tolist()}, attention list")
        display(df_attension.sort_values(f'atten_from{i}', ascending=False).iloc[:,[0,1,2,3+i]],)
    return df_ranking, df_attension

def check_killer(_title, _victim_name, _killer_name, _last_index, _story_len):
    return main_func01(_title, _victim_name, _killer_name, KILL, _last_index, _story_len)

### SpeckledBand(まだらの紐)
Who killed Julia? (criminal & explanation)
被害者: Julia
犯人: Roylott
犯行に用いたもの: snake
犯行動機: 母の相続財産を独占したい

In [14]:
def do_SpeckledBand_pred():
    title = 'SpeckledBand'
    victim_name = 'Julia'
    killer_name = 'Roylott'
    last_index = 360
    story_len = 80

    df_ranking, df_attention = check_killer(title, victim_name, killer_name, last_index, story_len)
    return df_ranking, df_attention
do_SpeckledBand_pred()
pass

2023-02-03 22:53:26 - INFO - 2806554648.py - 12 - The pred ranking about SpeckledBand:Roylott is 0


,predicate,whom,subject,why,what,where
0,word.predicate:kill,AllTitle:Watson,SpeckledBand:Roylott,AllTitle:Holmes,AllTitle:Holmes,SpeckledBand:Roylott_s_bedroom
1,AllTitle:Holmes,AllTitle:Holmes,AllTitle:Holmes,SpeckledBand:Roylott,AllTitle:Watson,SpeckledBand:Roylott
2,word.predicate:go,SpeckledBand:Roylott,AllTitle:Watson,SpeckledBand:1883-04-02T04,ResidentPatient:Elderly_man,SpeckledBand:Julia_s_bedroom
3,word.predicate:think,AbbeyGrange:Lady_Brackenstall,AbbeyGrange:Stanley_Hopkins,AbbeyGrange:Stanley_Hopkins,AbbeyGrange:Stanley_Hopkins,AllTitle:Holmes
4,word.predicate:come,SpeckledBand:Helen,AbbeyGrange:Lady_Brackenstall,SpeckledBand:Helen,DateTime:1883-04-02T04:00:00,ResidentPatient:Elderly_man
5,word.predicate:send,DevilsFoot:Sterndale,ACaseOfIdentity:Sutherland,SpeckledBand:mansion_of_Roylott,SpeckledBand:Roylott,<bos_e>
6,word.predicate:enter,ACaseOfIdentity:Hosmer,DancingMen:Sentence_I,ResidentPatient:Elderly_man,AbbeyGrange:Lady_Brackenstall,SpeckledBand:mansion_of_Roylott
7,word.predicate:meet,AbbeyGrange:Stanley_Hopkins,DevilsFoot:Mortimer,AbbeyGrange:Jack_Croker,DevilsFoot:Sterndale,word.predicate:small
8,word.predicate:hear,SpeckledBand:mansion_of_Roylott,word.predicate:examine,SpeckledBand:Helen_s_bedroom,SpeckledBand:1883-04-02T04,AbbeyGrange:Stanley_Hopkins
9,word.predicate:want,SpeckledBand:1883-04-02T04,DevilsFoot:George,DevilsFoot:Sterndale,word.predicate:find,DevilsFoot:Sterndale


index=478, triple=['SpeckledBand:360', 'kgc:when', 'SpeckledBand:1883-04-02T04'], attention list


,head,relation,tail,atten_from478
312,SpeckledBand:331,kgc:what,SpeckledBand:323,0.008014
380,SpeckledBand:343,kgc:when,SpeckledBand:1883-04-02T04,0.007724
419,SpeckledBand:350,kgc:when,SpeckledBand:1883-04-02T04,0.007611
424,SpeckledBand:351,kgc:when,SpeckledBand:1883-04-02T04,0.007596
373,SpeckledBand:341,kgc:when,SpeckledBand:1883-04-02T04,0.007364
391,SpeckledBand:345,kgc:when,SpeckledBand:1883-04-02T04,0.007349
472,SpeckledBand:359,kgc:when,SpeckledBand:1883-04-02T04,0.007312
386,SpeckledBand:344,kgc:when,SpeckledBand:1883-04-02T04,0.007001
402,SpeckledBand:347,kgc:when,SpeckledBand:1883-04-02T04,0.006907
354,SpeckledBand:338,kgc:when,SpeckledBand:1883-04-02T04,0.006850


index=479, triple=['SpeckledBand:360', 'kgc:time', 'DateTime:1883-04-02T04:00:00'], attention list


...

,head,relation,tail,atten_from485
391,SpeckledBand:345,kgc:when,SpeckledBand:1883-04-02T04,0.011999
419,SpeckledBand:350,kgc:when,SpeckledBand:1883-04-02T04,0.010702
380,SpeckledBand:343,kgc:when,SpeckledBand:1883-04-02T04,0.010430
427,SpeckledBand:352,kgc:subject,AllTitle:Holmes,0.009862
386,SpeckledBand:344,kgc:when,SpeckledBand:1883-04-02T04,0.009752
424,SpeckledBand:351,kgc:when,SpeckledBand:1883-04-02T04,0.009216
313,SpeckledBand:331,kgc:when,SpeckledBand:1883-04-02T03,0.008832
354,SpeckledBand:338,kgc:when,SpeckledBand:1883-04-02T04,0.008545
307,SpeckledBand:330,kgc:when,SpeckledBand:1883-04-02T03,0.008372
312,SpeckledBand:331,kgc:what,SpeckledBand:323,0.008069


index=486, triple=['<mask_e>', 'kgc:what', '<mask_e>'], attention list


,head,relation,tail,atten_from486
391,SpeckledBand:345,kgc:when,SpeckledBand:1883-04-02T04,0.011070
427,SpeckledBand:352,kgc:subject,AllTitle:Holmes,0.010419
312,SpeckledBand:331,kgc:what,SpeckledBand:323,0.009878
313,SpeckledBand:331,kgc:when,SpeckledBand:1883-04-02T03,0.008994
419,SpeckledBand:350,kgc:when,SpeckledBand:1883-04-02T04,0.008635
424,SpeckledBand:351,kgc:when,SpeckledBand:1883-04-02T04,0.008369
380,SpeckledBand:343,kgc:when,SpeckledBand:1883-04-02T04,0.008217
472,SpeckledBand:359,kgc:when,SpeckledBand:1883-04-02T04,0.008156
429,SpeckledBand:352,kgc:what,SpeckledBand:dog_whip,0.007843
416,SpeckledBand:350,kgc:subject,SpeckledBand:dog_whip,0.007830


index=487, triple=['<mask_e>', 'kgc:where', '<mask_e>'], attention list


,head,relation,tail,atten_from487
484,<mask_e>,kgc:subject,<mask_e>,0.009165
391,SpeckledBand:345,kgc:when,SpeckledBand:1883-04-02T04,0.008535
376,SpeckledBand:343,kgc:subject,SpeckledBand:lanthanum,0.007966
416,SpeckledBand:350,kgc:subject,SpeckledBand:dog_whip,0.007661
427,SpeckledBand:352,kgc:subject,AllTitle:Holmes,0.007514
422,SpeckledBand:351,kgc:subject,SpeckledBand:dog_whip,0.007379
429,SpeckledBand:352,kgc:what,SpeckledBand:dog_whip,0.007327
380,SpeckledBand:343,kgc:when,SpeckledBand:1883-04-02T04,0.007251
389,SpeckledBand:345,kgc:subject,SpeckledBand:safe,0.007247
487,<mask_e>,kgc:where,<mask_e>,0.007128
